<h1 align="center"> Campeonato Brasileiro Seria A - 2003 a 2016</h1>
<h3 align="center">Utilização das técnicas de Machine Learning para predizer o time que irá ser campeão do brasileirão 2017.</h3>

<h3>1 - Estruturação do ambiente para carga dos dados</h3>

<p>Importação das bibliotecas para estruturação dos dados</p>

In [1]:
import pandas as pd
import numpy as np
import os
import re

<p>Criação da estrutura do dataframe que ira comportar os dados depois de tratados</p>

In [2]:
# criação do dataframe que será utilizado para incluir os dados tratados
colunas = ['Rodada','Hora','DataPartida','DiaSemana','Ano','Arena','TimeCasa','GolCasa','TimeVisitante','GolVisitante',
           'FinalCasa',
           'FinalVisitante']
df_campeonato = pd.DataFrame(columns=colunas)

In [3]:
# Função que irá auxiliar na separação dos dados
def split_values(line):
    """
    Descrição: Separação dos campos.
    Parâmetro: String
    Retorno: String processada com os dados separados
    """
    return line.split(' - ')

In [4]:
# Função responsável por encontrar o caracter expecifico e modificá-lo, pois é preciso executar isso se não dará erro nos
# processos seguintes
def replace_n(pattern, string, n, replacer):
    """
    Descrição: A separação entre as informações dos time e suas pontuações não estão padronizadas e com isso,
    surgiu a necessidade de encontrar qual é o separador do determinado arquivo e efetuar a substituição do mesmo
    para continuar com o processo de carga e tratamento do arquivo.
    Parâmetro: 
        pattern: Separador padrão
        string: Texto onde se encontra o separados
        n: Posição do separador que deseja substituir. No caso, é o valor de vezes que o mesmo se repete exemplo:
           
        string = 'São Paulo 3 - 0 Botafogo'
        replace_n('-', string, 1, 'x')
        
    Retorno: String processada com os dados devidamente tratados
    """
    newString = ""

    wanted_span = list(re.finditer(pattern, string))[n-1].span()[0]
    newString = string[:wanted_span] + replacer + string[wanted_span+1:]
        
    return newString

In [5]:
# Função para retirar acentuação
# Fonte : encurtador.com.br/orsAV
import unicodedata

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [6]:
# Função para executar a limpeza dos dados
def TratamentoTexto(txt):
    """
    Descrição: Validação e tratamento dos dados. Por conta dos diferentes formatos dos arquivos, houve a necessidade de 
    tratar os dados de forma pontual para alcançar a maior uniformidade.
    Parâmetro: String
    Retorno: String processada com os dados devidamente tratados
    """
    
    # Retira as pontuações e deixa o texto em caixa baixa
    texto_tratado = strip_accents(txt) #strip_accents(txt.lower())
    
    # Validações para os dias da semana
    if 'feira' in texto_tratado or 'ferira' in texto_tratado or 'sabado' in texto_tratado or 'domingo' in texto_tratado:
        
        texto_tratado = texto_tratado.replace('segunda-feira','segunda')
        texto_tratado = texto_tratado.replace('terca-feira','terca')
        texto_tratado = texto_tratado.replace('quarta-feira','quarta')
        texto_tratado = texto_tratado.replace('quinta-feira','quinta')
        texto_tratado = texto_tratado.replace('sexta-feira','sexta')
        
        texto_tratado = texto_tratado.replace('segunda-ferira','segunda')
        texto_tratado = texto_tratado.replace('terca-ferira','terca')
        texto_tratado = texto_tratado.replace('quarta-ferira','quarta')
        texto_tratado = texto_tratado.replace('quinta-ferira','quinta')
        texto_tratado = texto_tratado.replace('sexta-ferira','sexta')
        
        # texto_tratado = texto_tratado.replace('sabado','sabado')
        # texto_tratado = texto_tratado.replace('domingo','domingo')
        
        

    # Validações para os dias da semana para os arquivos cujo os dias estão cortados
    if 'sab' in texto_tratado or 'dom' in texto_tratado or 'seg' in texto_tratado or  'ter' in texto_tratado or 'qua' in texto_tratado or 'qui' in texto_tratado or 'sex' in texto_tratado:
        
        texto_tratado = texto_tratado.replace('seg ','segunda - ')
        texto_tratado = texto_tratado.replace('ter ','terca - ')
        texto_tratado = texto_tratado.replace('qua ','quarta - ')
        texto_tratado = texto_tratado.replace('qui ','quinta - ')
        texto_tratado = texto_tratado.replace('sex ','sexta - ')
        texto_tratado = texto_tratado.replace('sab ','sabado - ')
        texto_tratado = texto_tratado.replace('dom ','domingo - ')
   
    # Validações antes de 
    if 'atletico' in texto_tratado or 'athletico' in texto_tratado or 'america' in texto_tratado or 'vitoria' in texto_tratado or 'vasco' in texto_tratado:
        
        # Atlético Mineiro
        texto_tratado = texto_tratado.replace('atletico-mg','atleticoMg')
        texto_tratado = texto_tratado.replace('atletico - mg','atleticoMg')
        texto_tratado = texto_tratado.replace('atletico mg','atleticoMg')
        
        # Athletico Paranaense
        texto_tratado = texto_tratado.replace('atletico - pr','athleticoPr')
        texto_tratado = texto_tratado.replace('atletico-pr','athleticoPr')
        texto_tratado = texto_tratado.replace('athletico - pr','athleticoPr')
        texto_tratado = texto_tratado.replace('athletico-pr','athleticoPr')
        texto_tratado = texto_tratado.replace('atletico pr','athleticoPr')
        
        
        # América Mineiro
        texto_tratado = texto_tratado.replace('america - mg','americaMg')
        texto_tratado = texto_tratado.replace('america-mg','americaMg')
        texto_tratado = texto_tratado.replace('america mg','americaMg')
        
        
        # América Mineiro
        texto_tratado = texto_tratado.replace('america-rn','americaRn')
        texto_tratado = texto_tratado.replace('america - rn','americaRn')
        texto_tratado = texto_tratado.replace('america rn','americaRn')
        
        # Vitória-BA
        texto_tratado = texto_tratado.replace('vitoria-ba','vitoriaBa')
        texto_tratado = texto_tratado.replace('vitoria - ba','vitoriaBa')
        texto_tratado = texto_tratado.replace('vitoria ba','vitoriaBa')
        
        # Atlético-GO
        texto_tratado = texto_tratado.replace('atletico-go','atleticoGo')
        texto_tratado = texto_tratado.replace('atletico - go','atleticoGo')
        texto_tratado = texto_tratado.replace('atletico go','atleticoGo')
        
        texto_tratado = texto_tratado.replace('vasco da gama','vasco')
               
    texto_tratado = texto_tratado.replace('matchday','rodada')
    
    return texto_tratado

In [7]:
def loadData(df_data, yearFile):
    """
    Descrição: Recebe o dataframe e o anos. Como os arquivos podem ter layouts diferentes, houve a necessidade
    de criar regras para cada tipo de arquivo.
    Parâmetro: df_data, yearFile
    Retorno: Lista contendo um dicionário com as informações separadas
    """
    #Instânciação dos objetos
    list_data = []
    full_data = {'Rodada':0, 'Data':0, 'Info':0}
    rodadaAtual = ""
    rodadaAnterior = ""
    
    #Percorre cada linha do dataframe
    for line in range(df_data.shape[0]):
        
        #cria variável para caminhar entre as regras
        string = TratamentoTexto(df_data["Dados"].iloc[df.index[line]])
        
        #print(type(yearFile))
        # Regras para cada arquivo
        if yearFile == '2003' or yearFile == '2004' or yearFile == '2006':
            string = string
            #print('yearFile')

        elif yearFile == '2005':
            #print('entou 2005')
            if '/' in string or 'rodada' in string or '\n' == string:
                #print('Passou aqui 1')
                string = string
            else:
                string = replace_n('-',string,2,'x')
                #print('aqui garotão: '+ string)

        elif yearFile == '2011' or yearFile == '2012' or yearFile == '2013' or yearFile == '2014' or yearFile == '2015':
            #print('aqui')
            if '/' in string or 'rodada' in string or '\n' == string:
                string = string
                #print(string)
            else:
                #print('aqui garoto!')
                string = replace_n('-',string.replace('\t',''),1,'x')
                #print(string)
        
        # Recupera a primeira linha de cada conjunto
        if 'rodada' in string:
            rodada = string
            #print('rodada: ' + rodada)
            continue
        else:
            
            # Recuperação da informação de data e dia da Semana
            if '/' in string:
                data_dia_semana = string
                
                #Regras de validação para gravação das variáveis
                if rodadaAtual != rodadaAnterior:

                    rodadaAtual = data_dia_semana

                elif  rodadaAnterior == "":
                    rodadaAnterior = data_dia_semana
                
                #print('Rodada Atual: '+ rodadaAtual)
                #print('Rodada Anterior: '+ rodadaAnterior)
            else:
                #print('aqui meu filho:' + string)
                
                #if yearFile == 2005:
                    #info = string
                #else:
                info = split_values(string)
                
                # controle de qual data deve ir para a informação correta
                if rodadaAtual != '':
                    full_data = {'Rodada':rodada, 'Data':rodadaAtual, 'Info':info}
                else:
                    full_data = {'Rodada':rodada, 'Data':rodadaAnterior, 'Info':info}
                
                # Gravação dos dados
                list_data.append(full_data)
    #print(list_data)
    return list_data

<h1>Carga e limpeza dos dados de acordo com o tipo do arquivo</h1>

In [8]:
"""
Descrição: Segue passos de tratamento pontual de cada arquivo
    1 - A partir da pasta indicada na variável "folderPath", lista a quantidade de arquivos e utiliza para carregar um por um.
    2 - Todo arquivo carregado para para o dataframe "df"
        a - A coluna com as informações é renomeada para "Dados"
        b - É feita a limpeza dos espaços
        a - Todos os caracteres são postos em caixa baixa
    3 - Recupera pelo nome do arquivo o ano e utiliza também como um dos parâmetros da função "loadData"
    4 - Pelo retorno da função "loadData" que será uma lista, carrega o novo dataframe df_base
    5 - Inclui a coluna de "Ano" no dataframe df_base
    6 - Sobrescreve "rodada" por "", pois existem diferenças de diferenças dependendo do arquivo
    7 - Como cada arquivo tem sua particularidade, houve a necessidade de separa por ano assim, tratar pontualmente cada
        a partir de seu layout.

"""
# Variável para retorno dos dados
list_data = []
listFiles = []

# Criação de lista com todos os arquivo dentro do diretório
folderPath = "arquivos-brasileirao-serie-a"

listFiles = os.listdir(folderPath)
numberFiles = len(listFiles)

for f in range(numberFiles):
    
    #print(f)
    FilePath = folderPath+'/'+listFiles[f]
    
    # Cria data frame com os dados
    df = pd.read_csv(FilePath, header=None)
    
    # Alteração do nome da coluna
    df.columns = ["Dados"]
    
    # Limpeza dos espaços
    df.Dados = df.Dados.str.strip()
    
    # atualiza os caracteres para caixa baixa 
    df.Dados = df.Dados.str.lower()
    
    #Recupera o ano no nome do arquivo para efetuar a gravação junto com os dados
    yearFile = re.findall(r'\d+',FilePath)[0]
    
    # Tratamento dos dados e gravação na lista
    # Esse aqui é o bichão parceiro!!
    list_data = loadData(df, yearFile)
    
    # Criação do Dataframe
    df_base = pd.DataFrame(list_data)
    
    # Efetua a gravação do ano do arquivo que está sendo lido no dataframe
    df_base["Ano"] = yearFile
    
    # Ajuste o campo Rodada
    rpc_rodada1 = ' rodada'
    rpc_rodada2 = 'rodada '
    
    df_base.Rodada = df_base.Rodada.str.replace(rpc_rodada1,'')
    df_base.Rodada = df_base.Rodada.str.replace(rpc_rodada2,'')
        
    # Tratamento dos arquivos em particular
    if df_base["Ano"].loc[df_base["Ano"].str.contains('2007|2008|2009|2010|2011|2012|2013|2014|2015')].count() > 1:
    
        #print('aqui')
        df_base['Hora'] = np.NAN
        df_base['Jogo'] = df_base.Info.apply(lambda x: x[0])
        df_base['Arena'] = np.NAN

        #Apaga a coluna Info
        df_base.drop('Info',axis=1,inplace=True)

        #Tratamento expecifico para os arquivo a partir do ano de 2011
        if df_base["Ano"].loc[df_base["Ano"].str.contains('2011|2012|2013|2014|2015')].count() > 1:

            #Tratamento da Hora e dia da semana
            hora = df_base.Data.str.split(' - ', expand=True)
            hora.columns = ['DiaSemana', 'DataPartida']
            df_base = pd.concat([df_base, hora],axis=1)
            df_base.drop('Data',axis=1,inplace=True)

            #Tratamento da data e dia da semana
            df_base["DataPartida"] = df_base.DataPartida.str.replace(']','/')+df_base.Ano
            df_base["DiaSemana"] = df_base.DiaSemana.str.replace('[','')

            #Divisão dos times(casa e visitante)
            times = df_base.Jogo.str.split('x', expand=True, n =1)
            times.columns = ['TimeCasa', 'TimeVisitante']
            df_base = pd.concat([df_base, times],axis=1)
            df_base.drop('Jogo',axis=1,inplace=True)

            #Divisão da pontuação de cada time
            df_base['GolCasa'] = df_base.TimeCasa.apply(lambda x: re.findall('\d', x)[0])
            df_base['GolVisitante'] = df_base.TimeVisitante.apply(lambda x: re.findall('\d', x)[0])
            
            
        else:
            #Tratamento da Hora e dia da semana
            hora = df_base.Data.str.split('-', expand=True)
            hora.columns = ['DataPartida', 'DiaSemana']

            df_base = pd.concat([df_base, hora],axis=1)
            df_base.drop('Data',axis=1,inplace=True)

            #Divisão dos times(casa e visitante)
            times = df_base.Jogo.str.split('-', expand=True, n =1)
            times.columns = ['TimeCasa', 'TimeVisitante']

            df_base = pd.concat([df_base, times],axis=1)
            df_base.drop('Jogo',axis=1,inplace=True)

            #Divisão da pontuação de cada time
            df_base['GolCasa'] = df_base.TimeCasa.apply(lambda x: x[-1])
            df_base['GolVisitante'] = df_base.TimeVisitante.apply(lambda x: x[0])

    # Regra para os arquivos entre 2003 até 2006
    if df_base["Ano"].loc[df_base["Ano"].str.contains('2003|2004|2005|2006')].count() > 1:
        
        #print('aqui')
        
        #Removendo da lista os dados para salvar na coluna
        df_base['Hora'] = df_base.Info.apply(lambda x: x[0])
        df_base['Jogo'] = df_base.Info.apply(lambda x: x[1])
        df_base['Arena'] = df_base.Info.apply(lambda x: x[2])

        #Percorrer a coluna jogo e ajustar o separador das pontuaçãos
        varTotalRegistros = df_base.shape[0]        
        for x in range(varTotalRegistros):
            df_base['Jogo'][df_base.index[x]] = replace_n('x', df_base["Jogo"][df_base.index[x]], 1, '-')

        #Apaga a coluna Info
        df_base.drop('Info',axis=1,inplace=True)

        #Tratamento da Hora e dia da semana
        hora = df_base.Data.str.split('-', expand=True, n=1)
        hora.columns = ['DataPartida', 'DiaSemana']

        df_base = pd.concat([df_base, hora],axis=1)
        df_base.drop('Data',axis=1,inplace=True)

        #Divisão dos times(casa e visitante)
        times = df_base.Jogo.str.split('-', expand=True, n =2)
        times.columns = ['TimeCasa', 'TimeVisitante']

        df_base = pd.concat([df_base, times],axis=1)
        df_base.drop('Jogo',axis=1,inplace=True)

        #Divisão da pontuação de cada time
        df_base['GolCasa'] = df_base.TimeCasa.apply(lambda x: re.findall('\d', x)[0])
        df_base['GolVisitante'] = df_base.TimeVisitante.apply(lambda x: re.findall('\d', x)[0])

      #Tratamento expecifico para os arquivo a partir do ano de 2016
    elif df_base["Ano"].loc[df_base["Ano"].str.contains('2016')].count() > 1:            

        df_base['Hora'] = np.NAN
        df_base['TimeCasa'] = df_base.Info.apply(lambda x: x[0])
        df_base['TimeVisitante'] = df_base.Info.apply(lambda x: x[1])
        df_base['Arena'] = np.NAN

        #Apaga a coluna Info
        df_base.drop('Info',axis=1,inplace=True)
        
        #Tratamento da data e dia da semana
        df_base["DataPartida"] = np.NAN
        df_base["DiaSemana"] = np.NAN

        #Divisão da pontuação de cada time
        df_base['GolCasa'] = df_base.TimeCasa.apply(lambda x: re.findall('\d', x)[0])
        df_base['GolVisitante'] = df_base.TimeVisitante.apply(lambda x: re.findall('\d', x)[0])
        
    #Limpeza
    df_base['TimeCasa'] = df_base.TimeCasa.str.replace('\d', '')
    df_base['TimeVisitante'] = df_base.TimeVisitante.str.replace('\d', '')
    
    #Reorganizando DataFrame
    df_base = df_base[['Rodada','Hora','DataPartida','DiaSemana','Ano','Arena','TimeCasa','GolCasa','TimeVisitante','GolVisitante']]
    
    #Tratamento dos Resultados dos times
    casa = 'GolCasa'
    visitante = 'GolVisitante'

    condicaoCasa = (df_base[casa] > df_base[visitante], #Vitória
                    df_base[casa] == df_base[visitante],#Empate
                    df_base[casa] < df_base[visitante]) #Derrota

    condicaoVisitante = (df_base[visitante] > df_base[casa], #Vitória
                         df_base[visitante] == df_base[casa],#Empate
                         df_base[visitante] < df_base[casa]) #Derrota

    resultado = ['v', 'e', 'd']


    df_base['FinalCasa'] = np.select(condicaoCasa, resultado, default=np.nan)
    df_base['FinalVisitante'] = np.select(condicaoVisitante, resultado, default=np.nan)
    
    # Finaliza incluindo os dados em um novo dataframe
    df_campeonato = df_campeonato.append(df_base, ignore_index=True, sort = False)

    # Nome do arquivo que foi processado
    print("Nome do Arquivo processado: " + listFiles[f] + " -- Total de registros: " + str(df_base.shape[0]))

    # Deleta dataframa e a lista para zerar tudo
    # Descomentar depois de finalizar os testes
    del list_data
    del df_base
    del df
    
# Retirar os registros duplicacdos
df_campeonato = df_campeonato.drop_duplicates()


# Quantidade de registro até o momento
print("Quantidade total de registros: " + str(df_campeonato.shape[0]))

Nome do Arquivo processado: brasileirao-seriea-2003.txt -- Total de registros: 552
Nome do Arquivo processado: brasileirao-seriea-2004.txt -- Total de registros: 553
Nome do Arquivo processado: brasileirao-seriea-2005.txt -- Total de registros: 473
Nome do Arquivo processado: brasileirao-seriea-2006.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2007.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2008.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2009.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2010.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2011.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2012.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2013.txt -- Total de registros: 380
Nome do Arquivo processado: brasileirao-seriea-2014.txt -- Total de registros: 380
Nome

In [9]:
df_campeonato.query('Ano == "2016"').head()

,Rodada,Hora,DataPartida,DiaSemana,Ano,Arena,TimeCasa,GolCasa,TimeVisitante,GolVisitante,FinalCasa,FinalVisitante
5378,1,NaN,NaN,NaN,2016,NaN,flamengo,1,sport,0,v,d
5379,1,NaN,NaN,NaN,2016,NaN,palmeiras,4,athleticoPr,0,v,d
5380,1,NaN,NaN,NaN,2016,NaN,atleticoMg,1,santos,0,v,d
5381,1,NaN,NaN,NaN,2016,NaN,coritiba,1,cruzeiro,0,v,d
5382,1,NaN,NaN,NaN,2016,NaN,santa cruz,4,vitoriaBa,1,v,d


<p>Ajustes dos campos que serão utilizados para análises</p>

In [10]:
# Verifica se existem registros vazio
print (df_campeonato[pd.to_numeric(df_campeonato.GolCasa, errors='coerce').isnull()])
print (df_campeonato[pd.to_numeric(df_campeonato.GolVisitante, errors='coerce').isnull()])

# Conversão nas colunas 
# pd.to_numeric(df_campeonato['GolCasa'])

# Inclusão dos dados que faltaram
df_campeonato.at[2380,'GolCasa'] = 0
df_campeonato.at[2696,'GolCasa'] = 3
df_campeonato.at[2573,'GolVisitante'] = 3


     Rodada Hora DataPartida DiaSemana   Ano Arena     TimeCasa GolCasa  \
2380      5  NaN  07/06/2008    sabado  2008   NaN  ipatinga              
2696     36  NaN  23/11/2008   domingo  2008   NaN     sport              

     TimeVisitante GolVisitante FinalCasa FinalVisitante  
2380       nautico            0         d              v  
2696    atleticoMg            0         d              v  
     Rodada Hora DataPartida DiaSemana   Ano Arena TimeCasa GolCasa  \
2573     24  NaN  04/09/2008    quinta  2008   NaN  vasco         1   

     TimeVisitante GolVisitante FinalCasa FinalVisitante  
2573      cruzeiro                      v              d  


In [11]:
# conversão das colunas
df_campeonato['Ano'] = pd.to_numeric(df_campeonato['Ano'])
df_campeonato['GolCasa'] = pd.to_numeric(df_campeonato['GolCasa'])
df_campeonato['GolVisitante'] = pd.to_numeric(df_campeonato['GolVisitante'])

In [12]:
#np.absolute(df_campeonato['GolCasa'] - df_campeonato['GolVisitante'])
df_campeonato.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5757 entries, 0 to 5756
Data columns (total 12 columns):
Rodada            5757 non-null object
Hora              1958 non-null object
DataPartida       5378 non-null object
DiaSemana         5378 non-null object
Ano               5757 non-null int64
Arena             1958 non-null object
TimeCasa          5757 non-null object
GolCasa           5757 non-null int64
TimeVisitante     5757 non-null object
GolVisitante      5757 non-null int64
FinalCasa         5757 non-null object
FinalVisitante    5757 non-null object
dtypes: int64(3), object(9)
memory usage: 744.7+ KB


In [13]:
# Gravação do dataframe em um arquivo csv para utilização posterior
df_campeonato.to_csv('df_campeonato.csv',sep=';', encoding='utf-8', index=False)

In [14]:
del df_campeonato